データの読み込みと前処理を行うためのnotebookです。  
モデルの学習と予測にはここで処理をかけたデータを利用するようにして下さい。

## 必要なライブラリのimport

In [1]:
import warnings
import time
import sys
import datetime

import numpy as np
import pandas as pd

warnings.simplefilter(action='ignore', category=FutureWarning)

## データの読み込み

In [2]:
def reduce_mem_usage(df, verbose=True):
    """
    データフレームのメモリ使用量を減らす。

    Parameters
    ----------
    df : pd.DataFrame
        メモリ使用量を削減したいデータフレーム。
    verbose : bool, optional
        メモリ使用量の削減結果を出力するかどうか（デフォルトは True）。

    Returns
    -------
    pd.DataFrame
        メモリ使用量が削減されたデータフレーム。
    """

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


def binarize(df):
    """
    指定された列を二値化する。

    Parameters
    ----------
    df : pd.DataFrame
        二値化対象のデータフレーム。

    Returns
    -------
    pd.DataFrame
        二値化されたデータフレーム。
    """

    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y': 1, 'N': 0})
    return df


def read_data(input_file):
    """
    指定されたファイルからデータを読み込み、前処理を行う。

    Parameters
    ----------
    input_file : str
        読み込むデータファイルのパス。

    Returns
    -------
    pd.DataFrame
        前処理されたデータフレーム。
    """
    
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
    return df

In [45]:
train = read_data('../data/processed/processed20240625_train.csv')
test = read_data('../data/processed/processed20240625_test.csv')

new_transactions = pd.read_csv('../data/row/new_merchant_transactions.csv',
                               parse_dates=['purchase_date'])

historical_transactions = pd.read_csv('../data/row/historical_transactions.csv',
                                      parse_dates=['purchase_date'])

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

## 特徴量作成

In [5]:
def reduce_mem_usage(df, verbose=True):
    """
    データフレームのメモリ使用量を減らす。

    Parameters
    ----------
    df : pd.DataFrame
        メモリ使用量を削減したいデータフレーム。
    verbose : bool, optional
        メモリ使用量の削減結果を出力するかどうか（デフォルトは True）。

    Returns
    -------
    pd.DataFrame
        メモリ使用量が削減されたデータフレーム。
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [47]:
# データ準備
historical_transactions['purchase_date'] = pd.to_datetime(historical_transactions['purchase_date'])
new_transactions['purchase_date'] = pd.to_datetime(new_transactions['purchase_date'])

# メモリ使用量の削減
historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)

Mem. usage decreased to 1360.42 Mb (56.2% reduction)
Mem. usage decreased to 87.99 Mb (58.0% reduction)


In [49]:
import calendar
from datetime import datetime

# 日付を三角関数変換し、新しいデータフレームとして取得する関数
def get_trigonometrical_date(df):
    # df['purchase_date'] = pd.to_datetime(df['purchase_date'])

    # 年ごとの日数を計算
    years = df['purchase_date'].dt.year
    unique_years = years.unique()
    days_in_year_dict = {year: (datetime(year + 1, 1, 1) - datetime(year, 1, 1)).days for year in unique_years}
    days_in_year = years.map(days_in_year_dict).to_numpy()

    # 年の日数に基づいて日をエンコード
    day_of_year = df['purchase_date'].dt.dayofyear.to_numpy()
    day_of_year_sin = np.sin(2 * np.pi * day_of_year / days_in_year)
    day_of_year_cos = np.cos(2 * np.pi * day_of_year / days_in_year)

    # 曜日をエンコード
    day_of_week = df['purchase_date'].dt.weekday.to_numpy()
    day_of_week_sin = np.sin(2 * np.pi * day_of_week / 7)
    day_of_week_cos = np.cos(2 * np.pi * day_of_week / 7)

    # 一か月の周期をエンコード（各月の最大日数を使用）
    days_in_month = df['purchase_date'].apply(lambda date: calendar.monthrange(date.year, date.month)[1]).to_numpy()
    day_of_month = df['purchase_date'].dt.day.to_numpy()
    day_of_month_sin = np.sin(2 * np.pi * day_of_month / days_in_month)
    day_of_month_cos = np.cos(2 * np.pi * day_of_month / days_in_month)

    # 一日の周期（時刻）をエンコード
    hour_of_day = df['purchase_date'].dt.hour + df['purchase_date'].dt.minute / 60.0
    hour_of_day_sin = np.sin(2 * np.pi * hour_of_day / 24)
    hour_of_day_cos = np.cos(2 * np.pi * hour_of_day / 24)

    # 結果をデータフレームに追加
    date_trigonometry = pd.DataFrame({
        'day_of_year': day_of_year,
        'day_of_year_sin': day_of_year_sin,
        'day_of_year_cos': day_of_year_cos,
        'day_of_month': day_of_month,
        'day_of_month_sin': day_of_month_sin,
        'day_of_month_cos': day_of_month_cos,
        'day_of_week': day_of_week,
        'day_of_week_sin': day_of_week_sin,
        'day_of_week_cos': day_of_week_cos,
        'hour_of_day': hour_of_day,
        'hour_of_day_sin': hour_of_day_sin,
        'hour_of_day_cos': hour_of_day_cos
    })

    return date_trigonometry

In [53]:
new_transactions_sin = new_transactions.join(get_trigonometrical_date(new_transactions))
historical_transactions_sin = historical_transactions.join(get_trigonometrical_date(historical_transactions))

In [56]:
# メモリ使用量の削減
historical_transactions_sin = reduce_mem_usage(historical_transactions_sin)
new_transactions_sin = reduce_mem_usage(new_transactions_sin)

Mem. usage decreased to 1971.22 Mb (46.6% reduction)
Mem. usage decreased to 129.17 Mb (47.3% reduction)


In [61]:
historical_transactions

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,day_of_month,day_of_month_sin,day_of_month_cos,day_of_week,day_of_week_sin,day_of_week_cos,hour_of_day,hour_of_day_sin,hour_of_day_cos,purchase_month
115,0,C_ID_4e6213e9bc,88,0,0,A,842,M_ID_22c9cfa265,-10,-0.730379,...,7,0.994629,0.104553,4,-0.433838,-0.900879,12.968750,-0.250488,-0.968262,2017-04-01
132,0,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-5,-0.723782,...,17,-0.406738,-0.913574,6,-0.781738,0.623535,22.671875,-0.342041,0.939453,2017-09-01
148,0,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-5,-0.723782,...,17,-0.406738,-0.913574,6,-0.781738,0.623535,22.671875,-0.342041,0.939453,2017-09-01
168,0,C_ID_4e6213e9bc,333,0,0,A,605,M_ID_c2ae34c2ef,0,-0.664262,...,20,-0.975098,-0.222534,1,0.781738,0.623535,10.953125,0.271484,-0.962402,2018-02-01
213,0,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-7,-0.738132,...,8,0.998535,-0.050659,5,-0.975098,-0.222534,7.550781,0.918945,-0.394775,2017-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112210,0,C_ID_570d5b0d4f,117,0,0,A,367,M_ID_6523fec105,-3,-0.733384,...,29,-0.394287,0.918945,5,-0.975098,-0.222534,19.531250,-0.920410,0.390625,2017-07-01
29112213,0,C_ID_570d5b0d4f,117,0,0,A,367,M_ID_6523fec105,0,-0.733955,...,12,0.651367,-0.758789,3,0.433838,-0.900879,19.046875,-0.962402,0.271484,2017-10-01
29112230,0,C_ID_925880baa8,-1,1,1,B,550,M_ID_09d7a200d1,0,0.973312,...,15,0.101196,-0.994629,0,0.000000,1.000000,10.570312,0.366455,-0.930176,2018-01-01
29112306,0,C_ID_5a397de34c,-1,1,1,B,661,M_ID_fc7d7969c3,0,-0.745405,...,4,0.724609,0.688965,5,-0.975098,-0.222534,20.765625,-0.749023,0.662598,2017-03-01


In [161]:
# purchase_month列を追加

historical_transactions_sin['purchase_month'] = historical_transactions_sin['purchase_date'].dt.to_period('M').dt.to_timestamp()
new_transactions_sin['purchase_month'] = new_transactions_sin['purchase_date'].dt.to_period('M').dt.to_timestamp()
historical_transactions_sin['epoch'] = pd.DatetimeIndex(historical_transactions_sin['purchase_date']).astype(np.int64) * 1e-9
new_transactions_sin['epoch'] = pd.DatetimeIndex(new_transactions_sin['purchase_date']).astype(np.int64) * 1e-9


all_transactions = pd.concat([historical_transactions_sin, new_transactions_sin], axis=0).reset_index(drop=True)
# authorized_flagに基づいてデータを分割
authorized_transactions = historical_transactions_sin[historical_transactions_sin['authorized_flag'] == 1]
historical_transactions = historical_transactions_sin[historical_transactions_sin['authorized_flag'] == 0]
new_transactions = new_transactions_sin.copy()


In [162]:
all_transactions

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,day_of_month_sin,day_of_month_cos,day_of_week,day_of_week_sin,day_of_week_cos,hour_of_day,hour_of_day_sin,hour_of_day_cos,purchase_month,epoch
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,...,-0.866211,0.500000,6,-0.781738,0.623535,15.546875,-0.801270,-0.598145,2017-06-01,1.498405e+09
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-7,-0.733128,...,0.101196,-0.994629,5,-0.975098,-0.222534,12.164062,-0.043610,-0.999023,2017-07-01,1.500121e+09
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-6,-0.720386,...,0.968262,-0.250732,2,0.975098,-0.222534,22.062500,-0.484863,0.874512,2017-08-01,1.502316e+09
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,...,0.406738,0.913574,5,-0.975098,-0.222534,10.101562,0.477051,-0.878906,2017-09-01,1.504347e+09
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-11,-0.722865,...,0.897949,-0.440430,4,-0.433838,-0.900879,1.233398,0.317383,0.948242,2017-03-01,1.489108e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31075387,1,C_ID_1320dee851,142,0,0,A,309,M_ID_7754b67f3b,2,-0.701660,...,0.951172,0.309082,4,-0.433838,-0.900879,14.601562,-0.629395,-0.777344,2018-04-01,1.523025e+09
31075388,1,C_ID_f112aa3381,158,0,0,A,560,M_ID_da063195b7,2,-0.694336,...,0.988281,0.151367,2,0.975098,-0.222534,13.320312,-0.337891,-0.941406,2018-03-01,1.520429e+09
31075389,1,C_ID_bd97b86450,69,0,1,B,278,M_ID_9a9ccb6544,1,-0.621094,...,0.848633,0.528809,0,0.000000,1.000000,12.070312,-0.017456,-1.000000,2018-03-01,1.520251e+09
31075390,1,C_ID_c0513fd84f,130,0,0,A,367,M_ID_40c28d596f,1,-0.656738,...,0.968262,-0.250732,4,-0.433838,-0.900879,14.781250,-0.666016,-0.746094,2018-03-01,1.520607e+09


In [223]:
def value1(df, suf=None):
    valuelist = ['day_of_year', 'day_of_year_sin', 'day_of_year_cos',
            'day_of_month', 'day_of_month_sin', 'day_of_month_cos',
            'day_of_week', 'day_of_week_sin', 'day_of_week_cos',
            'hour_of_day', 'hour_of_day_sin', 'hour_of_day_cos', 'epoch']
    valuelistwc = ['card_id', 'day_of_year', 'day_of_year_sin', 'day_of_year_cos',
            'day_of_month', 'day_of_month_sin', 'day_of_month_cos',
            'day_of_week', 'day_of_week_sin', 'day_of_week_cos',
            'hour_of_day', 'hour_of_day_sin', 'hour_of_day_cos', 'epoch']
    kari_add_df = all_transactions.card_id.drop_duplicates()
    kari_add_df = pd.DataFrame(kari_add_df.reset_index(drop=True))
    kari_add_df[valuelist] = 0
    for mode in ['max', 'min', 'old', 'new']:
        if mode == 'max':
            amt_idx = df.groupby('card_id')['purchase_amount'].idxmax()
        elif mode == 'min':
            amt_idx = df.groupby('card_id')['purchase_amount'].idxmin()
        elif mode == 'old':
            amt_idx = df.groupby('card_id')['purchase_date'].idxmin()
        elif mode == 'new':
            amt_idx = df.groupby('card_id')['purchase_date'].idxmax()
        kari_add_df = pd.merge(kari_add_df, df.loc[amt_idx, valuelistwc], how='left', on='card_id', suffixes=['', ('_' + suf + mode)])
    return kari_add_df.drop(columns=valuelist)

In [225]:
# ユーザーごとの最高額、最低額、最初の決済、最後の決済の日時情報
add_df = all_transactions.card_id.drop_duplicates()
add_df = pd.merge(add_df,value1(all_transactions, 'all'), on='card_id', how='inner')
add_df = pd.merge(add_df,value1(authorized_transactions, 'auth'), on='card_id', how='inner')
add_df = pd.merge(add_df,value1(historical_transactions, 'hist'), on='card_id', how='inner')
add_df = pd.merge(add_df,value1(new_transactions, 'new'), on='card_id', how='inner')


In [226]:
add_df

,card_id,day_of_year_allmax,day_of_year_sin_allmax,day_of_year_cos_allmax,day_of_month_allmax,day_of_month_sin_allmax,day_of_month_cos_allmax,day_of_week_allmax,day_of_week_sin_allmax,day_of_week_cos_allmax,...,day_of_month_newnew,day_of_month_sin_newnew,day_of_month_cos_newnew,day_of_week_newnew,day_of_week_sin_newnew,day_of_week_cos_newnew,hour_of_day_newnew,hour_of_day_sin_newnew,hour_of_day_cos_newnew,epoch_newnew
0,C_ID_4e6213e9bc,15,0.255371,0.966797,15,0.101196,-0.994629,0,0.000000,1.000000,...,29.0,-0.207886,0.978027,6.0,-0.781738,0.623535,18.218750,-0.998535,0.056702,1.525026e+09
1,C_ID_5037ff576e,208,-0.425049,-0.905273,27,-0.724609,0.688965,3,0.433838,-0.900879,...,23.0,-0.900879,0.433838,4.0,-0.433838,-0.900879,15.281250,-0.757324,-0.652832,1.519399e+09
2,C_ID_0e171c1b48,100,0.988770,-0.150024,10,0.866211,-0.500000,1,0.781738,0.623535,...,28.0,-0.406738,0.913574,5.0,-0.975098,-0.222534,19.312500,-0.941406,0.337891,1.524943e+09
3,C_ID_48fb13e70f,158,0.409424,-0.912598,7,0.994629,0.104553,2,0.975098,-0.222534,...,2.0,0.394287,0.918945,4.0,-0.433838,-0.900879,12.296875,-0.078430,-0.997070,1.519993e+09
4,C_ID_fc8e41b9cf,192,-0.162842,-0.986816,11,0.791016,-0.612305,1,0.781738,0.623535,...,27.0,-0.587891,0.809082,4.0,-0.433838,-0.900879,12.953125,-0.246094,-0.969238,1.524834e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325535,C_ID_803aa0aed4,66,0.907227,0.421143,7,0.988281,0.151367,2,0.975098,-0.222534,...,19.0,-0.743164,-0.668945,3.0,0.433838,-0.900879,5.550781,0.993164,0.117554,1.524116e+09
325536,C_ID_62df280b20,261,-0.976074,-0.217773,18,-0.587891,-0.809082,0,0.000000,1.000000,...,30.0,-0.201294,0.979492,1.0,0.781738,0.623535,16.984375,-0.964844,-0.262939,1.517332e+09
325537,C_ID_e49b1996b0,72,0.945801,0.325439,13,0.485352,-0.874512,1,0.781738,0.623535,...,13.0,0.485352,-0.874512,1.0,0.781738,0.623535,10.765625,0.317383,-0.948242,1.520938e+09
325538,C_ID_2863d2fa95,20,0.337402,0.941406,20,-0.791016,-0.612305,4,-0.433838,-0.900879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# データの結合
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')

train = pd.merge(train, authorized, on='card_id', how='left')
test = pd.merge(test, authorized, on='card_id', how='left')

train = pd.merge(train, new, on='card_id', how='left')
test = pd.merge(test, new, on='card_id', how='left')

train = pd.merge(train, final_group, on='card_id', how='left')
test = pd.merge(test, final_group, on='card_id', how='left')

train = pd.merge(train, auth_mean, on='card_id', how='left')
test = pd.merge(test, auth_mean, on='card_id', how='left')

train = pd.merge(train, additional_fields, on='card_id', how='left')
test = pd.merge(test, additional_fields, on='card_id', how='left')

## 前処理終了後のデータの保存
- 基本的にモデルの学習・ハイパーパラメータチューニングを行う際にはここで作成した同じデータを使い回して下さい。
- 適宜前処理を変更した場合はファイル名を変えるなどして管理して下さい。

In [22]:
# データの保存
train.to_csv('../data/processed/processed20240614_train.csv',index=None)
test.to_csv('../data/processed/processed20240614_test.csv',index=None)